#### Imports

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import select, text
import datetime
import plotly
import plotly.graph_objs as go
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

## Read and transform data

##### Read data

In [2]:
engine = create_engine("sqlite+pysqlite:///../../data/data.db", echo=False)

with engine.connect() as conn:
    rows = conn.execute(text("SELECT COUNT(utc_timestamp) FROM markets_data")).first()[0]
    print(f"Reading {rows} rows of data")
    df = pd.read_sql_table("markets_data", con=conn, index_col="utc_timestamp")

Reading 111360 rows of data


#### Transform data

In [3]:
df.index = pd.to_datetime(df.index)
df["cet_timestamp"] = df.index.tz_localize("utc").tz_convert("CET")

pt = df.pivot_table(index="cet_timestamp", columns=["country"], values="price_eur")
pt.sample()

country,BG,GR,HU,RO
cet_timestamp,,,,
2021-11-12 17:00:00+01:00,317.87,317.87,299.28,317.87


## Plot data

#### Define elements

In [4]:
def create_frequency_dropdown() -> widgets.widgets.widget_selection.Dropdown:
    time_agg_list = [("Hourly", "h"), ("Daily", "d"),
                     ("Weekly", "w"), ("Monthly", "m"),
                     ("Quarterly", "q"), ("Yearly", "y")]
    return widgets.Dropdown(options=time_agg_list, value="d", description="Aggregation")

freq_widget = create_frequency_dropdown()

In [5]:
fig = go.FigureWidget(
    layout=dict(
        height=600, 
        title_text="Energy prices explorer",
        xaxis_title = "Period, CET",
        yaxis_title = "Price, EUR/MWh",
        showlegend = True
    )
)

countries_widget = widgets.SelectMultiple(options=pt.columns)
freq_widget = create_frequency_dropdown()
output_test = widgets.Output()

start_date = widgets.DatePicker(
    description='Start date',
    disabled=False,
    value=datetime.datetime(2023,1,2).date()
)
end_date = widgets.DatePicker(
    description='End date',
    disabled=False,
    value=datetime.datetime(2023,1,15).date()
)

#### Add interactivity

In [6]:
def on_change(change):
    """ Update figure on change in the controls"""
    # with output_test:
    #     print(change['new'], change)
    freq_str = freq_widget.value
    start_str = start_date.value.strftime("%Y-%m-%d")
    end_str = end_date.value.strftime("%Y-%m-%d")
    agg_data = pt[start_str:end_str].groupby(pd.Grouper(freq=freq_str)).mean().round(2)
    with fig.batch_update():
        fig.data = []
        columns = countries_widget.value
        [fig.add_scatter(y=agg_data[col], x=agg_data.index, name=col) for col in columns]

        
countries_widget.observe(on_change, names='value')
freq_widget.observe(on_change, names='value')
start_date.observe(on_change, names='value')
end_date.observe(on_change, names='value')

countries_widget.value = tuple(pt.columns) 
period_controls = widgets.VBox([start_date, end_date, freq_widget])
all_controls = widgets.HBox([period_controls, countries_widget])

#### Display plot

In [7]:
display(widgets.VBox([output_test, all_controls, fig]))